### Importing libraries

In [1]:
import pandas as pd
from pulp import *

### Make datafram

In [2]:
data = pd.read_excel('./Delivery truck trip data.xlsx')

/tmp/ipykernel_4156/3630388012.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  data = pd.read_excel('./Delivery truck trip data.xlsx')


### Distance matrix

In [3]:
warehouse_str = data["Origin_Location"].unique()
CustomerPoint_str = data["Destination_Location"].unique()
# sorting
warehouse_str.sort()
CustomerPoint_str.sort()
# distance matrix
costs = []

for w in warehouse_str:
    distance_row = []
    for c in CustomerPoint_str:
        try:
            distance = data[(data['Origin_Location'] == w) & (data['Destination_Location'] == c)]['TRANSPORTATION_DISTANCE_IN_KM'].tolist()[0]
        except:
            distance = 10000000000000
        distance_row.append(distance)
    costs.append(distance_row)

### Get supply and demand

In [4]:
supply_str = dict(data["Origin_Location"].value_counts().sort_index())
demand_str = dict(data["Destination_Location"].value_counts().sort_index())

### Assigning Ids

In [5]:
supply = {}
demand = {}
warehouse = []
CustomerPoint = []

i = 0
for s in supply_str:
    supply[str(i)] = supply_str[s]
    warehouse.append(str(i))
    i+=1

i = 0
for s in demand_str:
    demand[str(i)] = demand_str[s]
    CustomerPoint.append(str(i))
    i+=1

### Distance to dictionary

In [6]:
# The cost data is made into a dictionary
costs = makeDict([warehouse,CustomerPoint],costs,0)

### Current Supply model

In [12]:
data["Route"] = data['Origin_Location'].astype(str) +" to "+ data["Destination_Location"]
current_routes = dict(data["Route"].value_counts().sort_index())

### Cuurent supply chain map

In [ ]:
for route in current_routes:
    print(route)

### Get cities

In [77]:
cities = [[],[],[]]

# All sources
for a in [route.split(' to ')[0] for route in current_routes]:
    if a not in cities[0]:
        cities[0].append(a)
        cities[1].append(data[data['Origin_Location'] == a]['Org_lat_lon'].unique()[0].split(',')[0])
        cities[2].append(data[data['Origin_Location'] == a]['Org_lat_lon'].unique()[0].split(',')[1])

# All destinations
for a in [route.split(' to ')[1] for route in current_routes]:
    if a not in cities[0]:
        cities[0].append(a)
        cities[1].append(data[data['Destination_Location'] == a]['Des_lat_lon'].unique()[0].split(',')[0])
        cities[2].append(data[data['Destination_Location'] == a]['Des_lat_lon'].unique()[0].split(',')[1])

### Current routes (Map)

In [98]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergeo(
        lon = cities[2],
        lat = cities[1],
        text = cities[0],
        textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":11},
        textposition="top center",
        name = "Candidate Facility",
        mode ="markers+text",
        marker = dict(
            size = 10,
            color = "black",
            line_color='black',
            line_width=0.5,
            sizemode = 'area')))


for a in [route.split(' to ') for route in current_routes]:
    o_lat = data[data['Origin_Location'] == a[0]]['Org_lat_lon'].unique()[0].split(',')[0]
    o_lon = data[data['Origin_Location'] == a[0]]['Org_lat_lon'].unique()[0].split(',')[1]
    d_lat = data[data['Destination_Location'] == a[1]]['Des_lat_lon'].unique()[0].split(',')[0]
    d_lon = data[data['Destination_Location'] == a[1]]['Des_lat_lon'].unique()[0].split(',')[1]
    fig.add_trace(go.Scattergeo(
        lat = [o_lat, d_lat], 
        lon = [o_lon, d_lon],
        mode = 'lines',
        line = dict(width = 1.5, color = 'blue'),
    ))

fig.update_layout(width=900, height=750,
        showlegend = False,
        geo = dict(
            scope = 'asia',
            landcolor = 'rgb(217, 217, 217)'))

### Optimize routes

In [101]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Routes Optimaization",LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in warehouse for b in CustomerPoint]

# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(warehouse,CustomerPoint),0,None,LpInteger)

prob += lpSum([vars[w][b]*costs[int(w)][int(b)] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in warehouse:
    prob += lpSum([vars[w][b] for b in CustomerPoint])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w

# The demand minimum constraints are added to prob for each demand node (customer)
# These constraints are stored for resolve later
customer_demand_constraint = {}
for b in CustomerPoint:
    constraint = lpSum([vars[w][b] for w in warehouse])>=demand[b]
    prob += constraint, "Sum_of_Products_into_customer_%s"%b
    customer_demand_constraint[b] = constraint


prob.writeLP("ProductDistributionProblem.lp") # The problem data is written to an .lp file

prob.solve()

/media/usama/Programming/Code files/assignment/venv/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning:

Spaces are not permitted in the name. Converted to '_'



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /media/usama/Programming/Code files/assignment/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b7acc3b874c54e3ca5416d8097d1a630-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b7acc3b874c54e3ca5416d8097d1a630-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 705 COLUMNS
At line 375107 RHS
At line 375808 BOUNDS
At line 469410 ENDATA
Problem MODEL has 700 rows, 93601 columns and 187200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 1.50 seconds
Cgl0004I processed model has 700 rows, 93600 columns (93600 integer (21320 of which binary)) and 187200 elements
Cbc0045I No integer variables out of 93600 objects (93600 integer) have costs
Cbc0045I branch on satisfied N create fake objective Y random cost Y
Cbc0012I Integer solution of 0 found by DiveCoeff

1

In [ ]:
for v in prob.variables():
    try:
        if v.varValue > 0:
            print(warehouse_str[int(str(v.name).split('_')[1])], 'to' ,CustomerPoint_str[int(str(v.name).split('_')[2])], "=", v.varValue)
    except:
        pass

### OptiMize Cell Routes

In [102]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergeo(
        lon = cities[2],
        lat = cities[1],
        text = cities[0],
        textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":11},
        textposition="top center",
        name = "Candidate Facility",
        mode ="markers+text",
        marker = dict(
            size = 10,
            color = "black",
            line_color='black',
            line_width=0.5,
            sizemode = 'area')))


for v in prob.variables():
    try:
        if v.varValue > 0:
            o_lat = data[data['Origin_Location'] == warehouse_str[int(str(v.name).split('_')[1])]]['Org_lat_lon'].unique()[0].split(',')[0]
            o_lon = data[data['Origin_Location'] == warehouse_str[int(str(v.name).split('_')[1])]]['Org_lat_lon'].unique()[0].split(',')[1]
            d_lat = data[data['Destination_Location'] == CustomerPoint_str[int(str(v.name).split('_')[2])]]['Des_lat_lon'].unique()[0].split(',')[0]
            d_lon = data[data['Destination_Location'] == CustomerPoint_str[int(str(v.name).split('_')[2])]]['Des_lat_lon'].unique()[0].split(',')[1]
            fig.add_trace(go.Scattergeo(
                lat = [o_lat, d_lat], 
                lon = [o_lon, d_lon],
                mode = 'lines',
                line = dict(width = 1.5, color = 'blue'),
            ))
    except:
        pass

fig.update_layout(width=900, height=750,
        showlegend = False,
        geo = dict(
            scope = 'asia',
            landcolor = 'rgb(217, 217, 217)'))